# Flights data cleaning
Erica Landreth

## Filtering to relevant rows/columns

In [0]:
# Airline Data (1 year)
df_flights = spark.read.parquet(f"dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_airlines_data_1y/")
shape_orig = (df_flights.count(), len(df_flights.dtypes))
display(df_flights)
print(f"Original shape: {shape_orig}")

In [0]:
## define columns to drop
# columns related to diversion: not enough data to use the diversion info
div_cols = [col for col in df_flights.columns if col.startswith('DIV') and col != "DIVERTED"]
# redundant carrier ID's (EDA indicated that OP_UNIQUE_CARRIER is sufficient)
xtra_carrier_cols = ["OP_CARRIER_AIRLINE_ID","OP_CARRIER"]
# redundant airport ID's (EDA indicated that ORIGIN/DEST and *_AIRPORT_SEQ_ID are sufficient)
xtra_airport_cols = [ \
  "ORIGIN_AIRPORT_ID","ORIGIN_CITY_MARKET_ID","ORIGIN_STATE_ABR","ORIGIN_STATE_FIPS","ORIGIN_STATE_NM","ORIGIN_WAC", \
  "DEST_AIRPORT_ID","DEST_CITY_MARKET_ID","DEST_STATE_ABR","DEST_STATE_FIPS","DEST_STATE_NM","DEST_WAC"]
# redundant flight info (could be recreated if need be)
xtra_flight_cols = ["WHEELS_OFF","WHEELS_ON","FLIGHTS","ACTUAL_ELAPSED_TIME","DISTANCE_GROUP"]
# redundant delay status info (could be recreated if need be)
xtra_time_cols = ["DEP_TIME","DEP_DELAY_NEW","DEP_DEL15","DEP_DELAY_GROUP","ARR_TIME","ARR_DELAY_NEW","ARR_DEL15","ARR_DELAY_GROUP"]

## fields to keep
# core features: useful for ML features and/or feature engineering
core_feats = ["FL_DATE","OP_UNIQUE_CARRIER","TAIL_NUM","OP_CARRIER_FL_NUM","ORIGIN","DEST","CRS_DEP_TIME","DEP_DELAY","CRS_ARR_TIME","ARR_DELAY","CANCELLED","DIVERTED","CRS_ELAPSED_TIME","AIR_TIME","DISTANCE"]
# we may or may not end up using these, but they can't easily be recreated later, so we'll keep them to be cautious
on_the_fence = ["ORIGIN_AIRPORT_SEQ_ID","DEST_AIRPORT_SEQ_ID","TAXI_OUT","TAXI_IN"]
# useful for time series analysis
time_series = ["QUARTER","MONTH","DAY_OF_MONTH","DAY_OF_WEEK","DEP_TIME_BLK","ARR_TIME_BLK","YEAR"]
# useful to sanity check that joins are successful
sanity_check = ["ORIGIN_CITY_NAME","DEST_CITY_NAME"]
# provides reasoning for cancellations, delays, and returns to gate
delay_info = [col for col in df_flights.columns if col.endswith("_DELAY") and col not in core_feats] + ["CANCELLATION_CODE"] + ["FIRST_DEP_TIME","LONGEST_ADD_GTIME","TOTAL_ADD_GTIME"]
    # Note: cancellation codes are: "A" for carrier-caused, "B" for weather, "C" for National Aviation System, and "D" for security

all_cols = div_cols+xtra_carrier_cols+xtra_airport_cols+xtra_flight_cols+xtra_time_cols+core_feats+on_the_fence+time_series+sanity_check+delay_info

missing = [col for col in df_flights.columns if col not in all_cols]
print(missing)

In [0]:
# define columns to keep
keep_me = core_feats + on_the_fence + time_series + sanity_check + delay_info
keep_me

In [0]:
# filter to columns of interest, and de-dupe
df_flights = df_flights.select(keep_me).distinct()

shape_filt = (df_flights.count(), len(df_flights.dtypes))
display(df_flights)
print(f"Filtered shape: {shape_filt}")

In [0]:
# sanity check: we expect half the records after de-dupe
shape_orig[0]/shape_filt[0] == 2.0

## Converting flight data to UTC time zones

### Create time zone reference file

The below code was used to look up the time zone for each airport. The resulting time zone info was saved out to parquet, so from this point on, just load the time zone parquet (see below for path).

In [0]:
# !pip install timezonefinder

In [0]:
# # imports
# from pyspark.sql.functions import udf
# from pyspark.sql.types import StringType
# from timezonefinder import TimezoneFinder
# import pytz
# from datetime import datetime
# from pyspark.sql.functions import col

# # load stations data
# df_stations = spark.read.parquet(f"dbfs:/mnt/mids-w261/datasets_final_project_2022/stations_data/stations_with_neighbors.parquet/")

# # get unique airport info from stations table
# df_locs = df_stations.select('neighbor_call','neighbor_lat','neighbor_lon').distinct()
# display(df_locs)

# # define function to look up time zones
# def find_timezone(lat, lng):
#     tf = TimezoneFinder()
#     timezone_str = tf.timezone_at(lat=lat, lng=lng)
#     return timezone_str if timezone_str else "Unknown"

# # define udf for time zone lookup
# find_timezone_udf = udf(find_timezone, StringType())

# # add time zone column
# df_locs = df_locs.withColumn("timezone", find_timezone_udf(col("neighbor_lat"), col("neighbor_lon")))

# # save df_time zone info as a parquet file
# folder_path = "dbfs:/student-groups/Group_4_1"
# df_locs.write.parquet(f"{folder_path}/external/tz_lookup.parquet")

In [0]:
# from pyspark.sql import Row

# df_tz = spark.read.parquet(f"dbfs:/student-groups/Group_4_1/external/tz_lookup.parquet")

# # manually fill in missing time zone info
# # note: neighbor_call is ICAO
# BBG = Row(neighbor_call="BBG", neighbor_lat=36.53856729627892, neighbor_lon=-93.19908127077512, timezone="America/Chicago")
# KOGS = Row(neighbor_call="KOGS", neighbor_lat=44.6820707679313, neighbor_lon=-75.47692203483886, timezone="America/New_York")
# NSTU = Row(neighbor_call="NSTU", neighbor_lat=-14.329024376251269, neighbor_lon=-170.71329690482548, timezone="Pacific/Pago_Pago")
# PGSN = Row(neighbor_call="PGSN", neighbor_lat=15.11974288544001, neighbor_lon=145.7282788950688, timezone="Pacific/Saipan")
# PGUM = Row(neighbor_call="PGUM", neighbor_lat=13.48562402083883, neighbor_lon=144.8001485238768, timezone="Pacific/Guam")
# TJPS = Row(neighbor_call="TJPS", neighbor_lat=18.01055087987774, neighbor_lon=-66.56323216254391, timezone="America/Puerto_Rico")
# TJSJ = Row(neighbor_call="TJSJ", neighbor_lat=18.457160454103658, neighbor_lon=-66.0974759565605, timezone="America/Puerto_Rico")
# US_0571 = Row(neighbor_call="US-0571", neighbor_lat=48.25780621107438, neighbor_lon=-103.74169879360201, timezone="America/Chicago")

# man_df = spark.createDataFrame([BBG,KOGS,NSTU,PGSN,PGUM,TJPS,TJSJ,US_0571])
# df_tz = df_tz.union(man_df)


In [0]:
# folder_path = "dbfs:/student-groups/Group_4_1"
# df_tz.write.mode('overwrite').parquet(f"{folder_path}/external/tz_lookup_manually_adjusted.parquet")

### Apply time zones to flight times for a small sample

In [0]:
# load data
df_airports = spark.read.option("header","true").csv(f"dbfs:/mnt/mids-w261/airport-codes_csv.csv")
df_flights = spark.read.parquet(f"dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_airlines_data_3m/")
df_tz = spark.read.parquet(f"dbfs:/student-groups/Group_4_1/external/tz_lookup_manually_adjusted.parquet")

In [0]:
# start with a very, very small flight data sample
tmp_flights = df_flights.limit(10)

First, we start with some joining, to get the relevant airport codes, and to introduce the time zone info, for both origin and destination airports.

In [0]:
# create temporary views
tmp_flights.createOrReplaceTempView("tmp_flights")
df_airports.createOrReplaceTempView("df_airports")
df_tz.createOrReplaceTempView("df_tz")

In [0]:
query = """

WITH origin AS(
SELECT  tf.FL_DATE as date,
        tf.CRS_DEP_TIME as dep_time,
        tf.CRS_ARR_TIME as arr_time,
        tf.ORIGIN as origin_iata,
        tf.DEST as dest_iata,
        a.ident as origin_icao
FROM tmp_flights as tf
LEFT JOIN df_airports as a on tf.ORIGIN = a.iata_code),

origin_dest AS(
SELECT  origin.date,
        origin.dep_time,
        origin.arr_time,
        origin.origin_iata,
        origin.dest_iata,
        origin.origin_icao,
        a.ident as dest_icao
FROM origin
LEFT JOIN df_airports as a on origin.dest_iata = a.iata_code),

origin_dest_tz1 AS(
SELECT  od.date,
        od.dep_time,
        od.arr_time,
        od.origin_iata,
        tz.timezone as origin_tz,
        od.dest_iata,
        od.origin_icao,
        od.dest_icao
FROM origin_dest as od
LEFT JOIN df_tz as tz on od.origin_icao = tz.neighbor_call
),

origin_dest_tz2 AS(
SELECT  od.date,
        od.dep_time,
        od.arr_time,
        od.origin_iata,
        od.origin_tz,
        od.dest_iata,
        tz.timezone as dest_tz,
        od.origin_icao,
        od.dest_icao
FROM origin_dest_tz1 as od
LEFT JOIN df_tz as tz on od.dest_icao = tz.neighbor_call
)

SELECT *
FROM origin_dest_tz2

"""

out = spark.sql(query)
display(out)

Now we apply the time zones to the departure and arrival times, and convert to UTC.

In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import pytz
from datetime import datetime
from pyspark.sql.functions import col

def create_datetime(yyyymmdd, hhmm, tz):
    """
    Create UTC timestamp from flights table columns
    yyyymmdd = FL_DATE
    hhmm = CRS_DEP_TIME or CRS_ARR_TIME
    tz = time zone from time zone table

    Returns UTC time stamp, (cast to string)
    """

    yyyy,MM,dd = yyyymmdd.split('-')
    yyyy = int(yyyy) # get year
    MM = int(MM) # get month
    dd = int(dd) # get day
    hh = hhmm//100 # get hour
    mm = hhmm%100 # get minute

    # create datetime variable
    dt = datetime(yyyy,MM,dd,hh,mm)
    # apply local time zone
    dt_local = pytz.timezone(tz).localize(dt)
    # convert to UTC
    dt_utc = dt_local.astimezone(pytz.utc)

    # return UTC datetime, cast to string
    return str(dt_utc)

# create_datetime('2015-02-27',901,'America/Chicago')

dt_udf = udf(create_datetime)
out = out.withColumn("dep_datetime", dt_udf(col("date"), col("dep_time"), col("origin_tz"))) \
    .withColumn("arr_datetime", dt_udf(col("date"), col("arr_time"), col("dest_tz")))
display(out)

# TODO: handle flights crossing midnight !!!

## Apply time zones to create full cleaned flights table

In [0]:
df_airports = spark.read.option("header","true").csv(f"dbfs:/mnt/mids-w261/airport-codes_csv.csv")
df_tz = spark.read.parquet(f"dbfs:/student-groups/Group_4_1/external/tz_lookup_manually_adjusted.parquet")

# start with a very, very small flight data sample
tmp_flights = df_flights.limit(10)

# create temporary views
df_flights.createOrReplaceTempView("df_flights")
df_airports.createOrReplaceTempView("df_airports")
df_tz.createOrReplaceTempView("df_tz")

In [0]:
# define all columns of the flights table
flights_cols = ", ".join(df_flights.columns)

query = f"""

WITH origin AS(
SELECT  {flights_cols},
        flights.FL_DATE as date,
        flights.CRS_DEP_TIME as dep_time,
        flights.CRS_ARR_TIME as arr_time,
        flights.ORIGIN as origin_iata,
        flights.DEST as dest_iata,
        a.ident as origin_icao
FROM df_flights as flights
LEFT JOIN df_airports as a on flights.ORIGIN = a.iata_code),

origin_dest AS(
SELECT  {flights_cols},
        origin.date,
        origin.dep_time,
        origin.arr_time,
        origin.origin_iata,
        origin.dest_iata,
        origin.origin_icao,
        a.ident as dest_icao
FROM origin
LEFT JOIN df_airports as a on origin.dest_iata = a.iata_code),

origin_dest_tz1 AS(
SELECT  {flights_cols},
        od.date,
        od.dep_time,
        od.arr_time,
        od.origin_iata,
        tz.timezone as origin_tz,
        od.dest_iata,
        od.origin_icao,
        od.dest_icao
FROM origin_dest as od
LEFT JOIN df_tz as tz on od.origin_icao = tz.neighbor_call
),

origin_dest_tz2 AS(
SELECT  {flights_cols},
        od.date,
        od.dep_time,
        od.arr_time,
        od.origin_iata,
        od.origin_tz,
        od.dest_iata,
        tz.timezone as dest_tz,
        od.origin_icao,
        od.dest_icao
FROM origin_dest_tz1 as od
LEFT JOIN df_tz as tz on od.dest_icao = tz.neighbor_call
)

SELECT *
FROM origin_dest_tz2

"""

out = spark.sql(query)
display(out)

In [0]:
tmp = out.filter( (col("origin_tz").isNull()) | (col("dest_tz").isNull()) )
display(tmp)

In [0]:
# from pyspark.sql.functions import udf
# from pyspark.sql.types import StringType
# import pytz
# from datetime import datetime, timedelta
# from pyspark.sql.functions import col

# def create_datetime(yyyymmdd, hhmm, tz):
#     """
#     Create UTC timestamp from flights table columns
#     yyyymmdd = FL_DATE
#     hhmm = CRS_DEP_TIME or CRS_ARR_TIME
#     tz = time zone from time zone table

#     Returns UTC time stamp, (cast to string)
#     """

#     yyyy,MM,dd = yyyymmdd.split('-')
#     yyyy = int(yyyy) # get year
#     MM = int(MM) # get month
#     dd = int(dd) # get day
#     hh = hhmm//100 # get hour
#     mm = hhmm%100 # get minute

#     if hh == 24:
#         hh = 0
#         do_shift = True
#     else:
#         do_shift = False

#     # create datetime variable
#     dt = datetime(yyyy,MM,dd,hh,mm)
#     if do_shift:
#         dt += timedelta(days=1)
#     # apply local time zone
#     dt_local = pytz.timezone(tz).localize(dt)
#     # convert to UTC
#     dt_utc = dt_local.astimezone(pytz.utc)

#     # return UTC datetime, cast to string
#     return str(dt_utc)

# dt_udf = udf(create_datetime)

# def create_arr_datetime(dep_yyyymmdd, dep_hhmm, dep_tz, flight_dur):
#     """
#     Create UTC timestamp from flights table columns
#     yyyymmdd = FL_DATE
#     hhmm = CRS_DEP_TIME or CRS_ARR_TIME
#     tz = time zone from time zone table

#     Returns UTC time stamp, (cast to string)
#     """

#     if flight_dur is None:
#         return None
    

#     yyyy,MM,dd = dep_yyyymmdd.split('-')
#     yyyy = int(yyyy) # get year
#     MM = int(MM) # get month
#     dd = int(dd) # get day
#     hh = dep_hhmm//100 # get hour
#     mm = dep_hhmm%100 # get minute

#     if hh == 24:
#         hh = 0
#         do_shift = True
#     else:
#         do_shift = False

#     # create datetime variable
#     dt = datetime(yyyy,MM,dd,hh,mm)
#     if do_shift:
#         dt += timedelta(days=1)
#     # apply local time zone
#     dt_local = pytz.timezone(dep_tz).localize(dt)
#     # convert to UTC
#     dt_utc = dt_local.astimezone(pytz.utc)
#     dt_arr_utc = dt_utc + timedelta(minutes=flight_dur)

#     # return UTC datetime, cast to string
#     return str(dt_arr_utc)
    
# dt_arr_udf = udf(create_arr_datetime)

# def create_datetime_v2(yyyymmdd, dep_hhmm, arr_hhmm, tz):
#     """
#     Create UTC timestamp from flights table columns
#     yyyymmdd = FL_DATE
#     hhmm = CRS_DEP_TIME or CRS_ARR_TIME
#     tz = time zone from time zone table

#     Returns UTC time stamp, (cast to string)
#     """

#     yyyy,MM,dd = yyyymmdd.split('-')
#     yyyy = int(yyyy) # get year
#     MM = int(MM) # get month
#     dd = int(dd) # get day
#     hh = arr_hhmm//100 # get hour
#     mm = arr_hhmm%100 # get minute

#     if hh == 24:
#         hh = 0
#         do_shift = True
#     else:
#         do_shift = False

#     # create datetime variable
#     dt = datetime(yyyy,MM,dd,hh,mm)
#     if do_shift:
#         dt += timedelta(days=1)
#     if dep_hhmm > arr_hhmm:
#         dt += timedelta(days=1)
#     # apply local time zone
#     dt_local = pytz.timezone(tz).localize(dt)
#     # convert to UTC
#     dt_utc = dt_local.astimezone(pytz.utc)

#     # return UTC datetime, cast to string
#     return str(dt_utc)

# dt2_udf = udf(create_datetime_v2)

# out = out.withColumn("dep_datetime", dt_udf(col("date"), col("dep_time"), col("origin_tz"))) \
#     .withColumn("arr_datetime_SANITYCHECK", dt2_udf(col("date"), col("dep_time"), col("arr_time"), col("dest_tz"))) \
#     .withColumn("arr_datetime", dt_arr_udf(col("date"), col("dep_time"), col("origin_tz"), col("CRS_ELAPSED_TIME")))

In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import pytz
from datetime import datetime, timedelta
from pyspark.sql.functions import col

def to_utc(yyyymmdd, dep_hhmm, arr_hhmm, dep_tz, arr_tz, flight_dur):
    """
    Create UTC timestamp from flights table columns
    yyyymmdd = FL_DATE
    hhmm = CRS_DEP_TIME or CRS_ARR_TIME
    tz = time zone from time zone table

    Returns UTC time stamp, (cast to string)
    """

    yyyy,MM,dd = yyyymmdd.split('-')
    yyyy = int(yyyy) # get year
    MM = int(MM) # get month
    dd = int(dd) # get day

    dep_hh = dep_hhmm//100 # get hour
    dep_mm = dep_hhmm%100 # get minute
    if dep_hh == 24:
        dep_hh = 0
        dep_shift = True
    else:
        dep_shift = False

    arr_hh = arr_hhmm//100 # get hour
    arr_mm = arr_hhmm%100
    if arr_hh == 24:
        arr_hh = 0
        arr_shift = True
    else:
        arr_shift = False

    # create datetime variable for departure
    dt_dep = datetime(yyyy,MM,dd,dep_hh,dep_mm)
    if dep_shift:
        dt_dep += timedelta(days=1)
    # apply local time zone
    dep_local = pytz.timezone(dep_tz).localize(dt_dep)
    # convert to UTC
    dep_utc = dep_local.astimezone(pytz.utc)

    # create datetime variable for arrival
    dt_arr = datetime(yyyy,MM,dd,arr_hh,arr_mm)
    if arr_shift:
        dt_arr += timedelta(days=1)
    # apply local time zone
    arr_local = pytz.timezone(arr_tz).localize(dt_arr)
    # convert to UTC
    arr_utc = arr_local.astimezone(pytz.utc)

    if dep_utc > arr_utc:
        arr_utc += timedelta(days=1)

    # sanity check
    arr_utc_SC = dep_utc + timedelta(minutes=flight_dur)

    # return UTC datetime, cast to string
    return (str(dep_utc), str(arr_utc), str(arr_utc_SC))

# to_utc('2015-02-27',901,1031,'America/Chicago','America/Los_Angeles',180)

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, StructType

schema = StructType([
    StructField("dep_datetime", StringType(), True),
    StructField("arr_datetime", StringType(), True),
    StructField("arr_datetime_SANITYCHECK", StringType(), True)
])

dt_udf = udf(to_utc, schema)

out = out.withColumn('processed', dt_udf(col("date"), col("dep_time"), col("arr_time"), col("origin_tz"), col("dest_tz"), col("CRS_ELAPSED_TIME")))

# out = out.withColumn("dep_datetime", dt_udf(col("date"), col("dep_time"), col("origin_tz"))) \
#     .withColumn("arr_datetime_SANITYCHECK", dt2_udf(col("date"), col("dep_time"), col("arr_time"), col("dest_tz"))) \
#     .withColumn("arr_datetime", dt_arr_udf(col("date"), col("dep_time"), col("origin_tz"), col("CRS_ELAPSED_TIME")))

In [0]:
cols = [c for c in out.columns if c != "processed"]
cols += ["processed.dep_datetime","processed.arr_datetime","processed.arr_datetime_SANITYCHECK"]
out = out.select(cols)
display(out)